In [2]:
import xml.etree.ElementTree as ET
import duckdb
from datetime import datetime
from duckdb import ConstraintException, FatalException
from pathlib import Path
from tqdm import tqdm

In [14]:
# Connect to sql database
con = duckdb.connect(database='german-parliament', read_only=False)

In [15]:
def create_db(reset_db=False):
    """
    Creates the necessary tables in the DuckDB database for storing plenary minutes.
    This function creates the 'party', 'speaker', and 'plenary_minute' tables, as well as sequences for party and speaker IDs.

    Args:
        reset_db (bool): If True, drops existing tables and sequences before creating new ones.
                          Defaults to False.

    Returns:
        None
    """
    if reset_db:
        con.execute("DROP TABLE IF EXISTS speech;")
        con.execute("DROP TABLE IF EXISTS speaker;")
        con.execute("DROP TABLE IF EXISTS party;")
        con.execute("DROP TABLE IF EXISTS plenary_minute;")
        con.execute("DROP TABLE IF EXISTS session;")


        con.execute("DROP SEQUENCE IF EXISTS party_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS speaker_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS legislative_period_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS session_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS speech_id_seq;")

    # Create a sequence for party IDs to ensure unique IDs for each party. It's like an auto-incrementing primary key.
    con.execute("CREATE SEQUENCE IF NOT EXISTS party_id_seq START 1;")
    # Create the party which will be used to identify a party
    con.execute(
        """
        CREATE TABLE IF NOT EXISTS party (
            id INTEGER DEFAULT nextval('party_id_seq') PRIMARY KEY,
            -- If we insert a party we just use DEFAULT to get the next value from the sequence.
            name VARCHAR UNIQUE NOT NULL
        );
        """
    )

    # Create a sequence for speaker IDs to ensure unique IDs for each speaker.
    con.execute("CREATE SEQUENCE IF NOT EXISTS speaker_id_seq START 1;")
    # Create the speaker which will be used to identify a speaker
    con.execute("""
        CREATE TABLE IF NOT EXISTS speaker (
            harmonised_id INTEGER DEFAULT nextval('speaker_id_seq') UNIQUE, -- harmonised_id is used to identify the speaker in the XML file of period < 19
            -- If we insert a speaker we just use DEFAULT to get the next value from the sequence.
            new_id INTEGER UNIQUE, -- new_id is used to identify the speaker in the XML file of period 19 and later. This id is provided by the Bundestag.
            name VARCHAR NOT NULL,
            party_id INTEGER REFERENCES party(id) NOT NULL,
            PRIMARY KEY (name, party_id)
        );
        """
    )

    # Create a sequence for legislative periods to ensure unique IDs for each legislative period.
    con.execute("CREATE SEQUENCE IF NOT EXISTS legislative_period_id_seq START 1;")
    # Create the session table to store information about legislative periods and sessions.
    con.execute("""
        CREATE TABLE IF NOT EXISTS session (
            id INTEGER DEFAULT nextval('legislative_period_id_seq') UNIQUE NOT NULL,
            legislative_period INTEGER NOT NULL,
            session_no INTEGER NOT NULL,
            date DATE NOT NULL,
            primary key (legislative_period, session_no)
        );
        """
    )

    # Create a sequence for plenary minute IDs to ensure unique IDs for each plenary minute.
    con.execute("CREATE SEQUENCE IF NOT EXISTS speech_id_seq START 1;")
    con.execute("""
        CREATE TABLE IF NOT EXISTS speech (
            id VARCHAR DEFAULT nextval('speech_id_seq') PRIMARY KEY,
            -- id is a unique identifier for each speech, generated from the sequence for speeches for periods < 19.
            -- For newer periods the bundestag provides a unique id for each speech. Those begin with "ID"
            session_id INTEGER REFERENCES session(id) NOT NULL,
            speaker_harmonised_id INTEGER REFERENCES speaker(harmonised_id), -- we can use harmonised_id, since this id is available for all periods
            role VARCHAR, -- Role of the speaker, e.g. 'mp'
            position VARCHAR, -- Position of the speaker, e.g. 'Präsident', 'Alterspräsident', etc. | In newer periods this is named Rolle!
            content TEXT NOT NULL
        );
        """
    )
    con.commit()

In [16]:
def parse_xml(file_path):
    """
    Parses an XML file and returns the root element.

    Args:
        file_path (str): The path to the XML file.

    Returns:
        xml.etree.ElementTree.Element: The root element of the parsed XML tree,
                                         or None if parsing fails.
    """
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        return root
    except ET.ParseError as e:
        print(f"Error parsing XML file: {e}")
        return None
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

In [24]:
def insert_session(legislative_period, session, date):
    """ Inserts session information into the session table.

    Args:
        meta_data (xml.etree.ElementTree.Element): The title statement element containing legislative period and session information, as well as the date.

    Raises:
        Exception: If the titleStmt is None or if the publicationStmt is not found in the XML file.

    Returns:
        None
    """
    # print("------------------------------------")
    # print(f"Legislative Period: {legislative_period}, Session: {session}")
    # print(f"Publication Date: {date}")#
    # print("------------------------------------\n")

    # Insert session into the session table(

    sql = """
        INSERT INTO session (legislative_period, session_no, date)
        VALUES (?, ?,?)
        ON CONFLICT DO NOTHING;
    """

    params = (
        legislative_period,
        session,
        date
    )
    con.execute(sql, params)

    # con.execute(f"""
    #     INSERT INTO session (id, legislative_period, session_no, date)
    #     VALUES (DEFAULT, '{legislative_period}', '{session}', '{date}');
    # """)


def insert_party(attributes):
    # Insert party into the party table if it does not exist. The on conflict do nothing clause ensures that our key constraints aren't violated
    sql = "INSERT INTO party (name) VALUES (?) ON CONFLICT DO NOTHING;"
    con.execute(sql, (attributes["party"],))


def insert_speaker(attributes):
    # If id is in attributes the speaker is from a period >= 19, otherwise it is from a period < 19 and we will use the old_id.
    if "id" in attributes:
        # Checks if speaker, party pair already exists in the speaker table, but the new_id is still NULL. -> The politician has been inserted in a < 19 period, but not in a later period.
        # This is done to match the id's from us with the id's assigned by the Bundestag for periods >= 19.
        sql = "SELECT COUNT(*) FROM speaker WHERE name = ? AND party_id = (SELECT id FROM party WHERE name = ?) AND new_id is NULL;"
        count = con.execute(sql, (attributes["name"], attributes["party"])).fetchone()[0]
        if count > 0:
            # If the speaker already exists, we update the new_id to the id provided by the Bundestag.
            sql = """
                UPDATE speaker
                SET new_id = ?
                WHERE name = ? AND party_id = (SELECT id FROM party WHERE name = ?);
            """
            con.execute(sql, (attributes["id"], attributes["name"], attributes["party"]))
            return
        else:
            # The speaker does not exist yet, so we insert it.
            sql = """
                INSERT INTO speaker (new_id, name, party_id)
                VALUES (
                    ?,                                    -- speaker’s new id
                    ?,                                    -- speaker’s full name
                    (SELECT id FROM party WHERE name = ?) -- look-up party once, safely
                )
                ON CONFLICT DO NOTHING;
                """
    else:
        # The old id gets assigned automatically by the database, so we don't need to provide it.
        sql = """
            INSERT INTO speaker (name, party_id)
            VALUES (
                ?,                                    -- speaker’s full name
                (SELECT id FROM party WHERE name = ?) -- look-up party once, safely
            )
            ON CONFLICT DO NOTHING;
            """

        con.execute(sql, (attributes["name"], attributes["party"]))




def insert_speech(speech, attributes, legislative_period, session, speech_id=None):
    # Insert speech into the speech table
    # print(attributes)
    speech_string = ET.tostring(speech, encoding='unicode') if speech is not None else ''

    party_id = con.execute(f"""
                    SELECT id FROM party WHERE name = '{attributes['party']}';
                """).fetchone()[0]
    speaker_id = con.execute(f"""
                    SELECT harmonised_id FROM speaker WHERE name = '{attributes['name']}' AND party_id = {party_id};
                """).fetchone()[0]
    session_id = con.execute(f"""
                    SELECT id FROM session WHERE legislative_period = '{legislative_period}' AND session_no = '{session}';
                 """).fetchone()[0]

    speaker_role = attributes.get("role")
    speaker_role = None if speaker_role == 'NA' else speaker_role  # Convert 'NA' to None for SQL NULL
    speaker_position = attributes.get("position")
    speaker_position = None if speaker_position == 'NA' else speaker_position  # Convert 'NA' to None for SQL NULL
    # values to bind — order must match the placeholders
    if speech_id is None:
        # If we have no speech_id, we know that this is a speech from a period < 19 and we can use our own, automatically generating id.
        sql = """
        INSERT INTO speech
               (session_id, speaker_harmonised_id, role, position, content)
        VALUES (?, ?, ?, ?, ?)
        ON CONFLICT DO NOTHING;
        """
        params = (
        session_id,
        speaker_id,
        speaker_role,
        speaker_position,
        speech_string
        )

        con.execute(sql, params)
    else:
        # If we have a speech_id, we know that this is a speech from a period >= 19 and we can insert the new_id speech id instead of using our proprietary id.
        sql = """
        INSERT INTO speech
               (id, session_id, speaker_harmonised_id, role, position, content)
        VALUES (?, ?, ?, ?, ?, ?)
        ON CONFLICT DO NOTHING;
        """
        params = (
            speech_id,
            session_id,
            speaker_id,
            speaker_role,
            speaker_position,
            speech_string
        )
        con.execute(sql, params)

def get_speaker_attributes(speaker_data) -> dict:
    """
    Extracts speaker attributes from the speaker data.

    Args:
        speaker_data (xml.etree.ElementTree.Element): The speaker data element.

    Returns:
        dict: A dictionary containing the speaker's name, party, and ID.
    """
    speaker_id = speaker.attrib.get('id')
    speaker_data = speaker.find('name')
    # Extract the speaker's first and last name
    speaker_first_name = speaker_data.find('vorname').text
    speaker_last_name = speaker_data.find('nachname').text
    speaker_full_name = speaker_first_name + " " + speaker_last_name
    # Extract the speakers party
    speaker_party = speaker_data.find('fraktion')
    if speaker_party is not None:
        speaker_party = speaker_party.text
    else:
        speaker_party = None
    # Return the speaker attributes as a dictionary
    return {"party": speaker_party, "name": speaker_full_name, "id": speaker_id}


def parse_old_plenary_minute(root):
    """ Parses the plenary minute XML root element and extracts speeches, speakers, and parties and inserts them into the database.

    Args:
        root (xml.etree.ElementTree.Element): The root element of the XML tree containing the data of a single plenary minute.

    Raises:
        Exception: If the XML root element is None.

    Returns:
        None

    """
    if root is not None:
        # Get the meta data information, i.e. date, session, etc.
        meta_data = root.find('teiHeader').find('fileDesc')
        
        titleStmt = meta_data.find('titleStmt')
        publicationStmt = meta_data.find('publicationStmt')
        
        legislative_period = titleStmt.find('legislativePeriod').text
        session = titleStmt.find('sessionNo').text
        date = publicationStmt.find('date').text

        # Insert session entry into the db
        insert_session(legislative_period, session, date)
        #Get the actual content of the plenary minute
        plenary_minute = root.find('text').find('body').findall('div')
        speeches = []
        # Get all 'div' elements in the 'body' of the 'text'. Those are the elements that contain the actual plenary minute content.
        for div in plenary_minute:
            speeches.extend(div.findall('sp'))

        for speech in speeches:
            # Get attributes of the speech element
            attributes = speech.attrib
            # Insert party of speaker into db
            insert_party(attributes)
            # Insert speaker (politician) into db
            insert_speaker(attributes)
            # Insert the speech into the db
            insert_speech(speech, attributes, legislative_period, session)
    else:
        raise Exception("Missing XML root element.")

def parse_new_plenary_minute(root):
    if root is not None:
        # Get metadata
        kopfdaten = root.find("vorspann").find("kopfdaten")
        # Get period and session no
        plenarprotokoll_nummer = kopfdaten.find("plenarprotokoll-nummer")
        legislative_period = plenarprotokoll_nummer.find("wahlperiode").text
        session_number = plenarprotokoll_nummer.find("sitzungsnr").text
        # Get the date
        raw_date = kopfdaten.find("veranstaltungsdaten").find('datum').attrib.get("date")
        date = datetime.strptime(raw_date, "%d.%m.%Y")
        # Insert the session into the database
        insert_session(legislative_period, session_number, date) #@ todo this will be the existing function
        # Get the session data where the speeches are stored
        tagesordnungspunkte = root.find("sitzungsverlauf").findall("tagesordnungspunkt")
        # Iterate over each Tagesordnungspunkt (agenda item)
        for top in tagesordnungspunkte:
            speeches = top.findall('rede')
            for speech in speeches:
                # This will extract the speaker data and feeds it into the database.
                speech_id = speech.attrib.get('id')
                speaker_data = speech.find('p').find('redner')
                attributes = get_speaker_attributes(speaker_data)
                insert_party(attributes) # passed as list to ensure compatibility with the insert function @todo existing function
                insert_speaker(attributes) #@ todo existing function
                insert_speech(speech, attributes, legislative_period, session_number, speech_id) #@todo existing function

def parse_legislative_period(path, period):
    print(f"Parsing legislative period at path: {path}")
    # We materialise the generator once, to obtain all entries, so we can call len() on it
    files = list(Path(path).rglob("*"))
    # tqdm is used to create a nice loading bar for progress
    for file in tqdm(files, total=len(files), desc="Processing files"):
        if file.suffix == ".xml":
            # print(f"Parsing file: {file}")
            root = parse_xml(file)
            # Depending on the period we parse the XML differently, since the structure of the XML changed after period 18.
            if period <= 18:
                parse_old_plenary_minute(root)
            elif period > 18:
                parse_new_plenary_minute(root)
    print(f"Finished parsing legislative period at path: {path}")



def main():
    # Create database tables
    create_db(True)
    # Parse each period between period 1 and 18
    for period in range(21, 22):
        # 02:d adjusts the number to be two digits, e.g. 01, 02, ..., 10, 11, ...
        try:
            con.execute("BEGIN TRANSACTION;")
            parse_legislative_period(f'../data/plenary_minutes/wahlperiode_{period:02d}')
            con.execute("COMMIT;")
        except FatalException as e:
            con.execute("ROLLBACK;")
            print(f"Error processing legislative period {period}: {e}")


if __name__ == "__main__":
    main()

# Always close connection to ensure that we won't have problems reconnecting to the db!
con.close()

Parsing legislative period at path: ../data/plenary_minutes/wahlperiode_01


Processing files:   5%|▌         | 15/282 [00:03<01:05,  4.05it/s]


KeyboardInterrupt: 

In [27]:
con.execute("select * from speech where position is null").fetchdf()

,id,session_id,speaker_old_id,speaker_new_id,role,position,content
0,4,1,4,<NA>,mp,None,"<sp who=""Kohl (Stuttgart)"" parliamentary_group..."
1,6,1,6,<NA>,mp,None,"<sp who=""Wellhausen"" parliamentary_group=""FDP""..."
2,8,1,8,<NA>,mp,None,"<sp who=""Horlacher"" parliamentary_group=""CSU"" ..."
3,10,1,10,<NA>,mp,None,"<sp who=""Gülich"" parliamentary_group=""SPD"" rol..."
4,12,1,12,<NA>,mp,None,"<sp who=""Ritzel"" parliamentary_group=""SPD"" rol..."
...,...,...,...,...,...,...,...
718,1693,16,591,<NA>,mp,None,"<sp who=""von Merkatz"" parliamentary_group=""DP""..."
719,1695,16,943,<NA>,mp,None,"<sp who=""Blachstein"" parliamentary_group=""SPD""..."
720,1697,16,943,<NA>,mp,None,"<sp who=""Blachstein"" parliamentary_group=""SPD""..."
721,1699,16,943,<NA>,mp,None,"<sp who=""Blachstein"" parliamentary_group=""SPD""..."


In [29]:
con.close()

In [ ]:
con.execute("select * from speaker join party on speaker.party_id = party.id ;").fetchdf()